In [1]:
import os
from rdkit import Chem

def modify_atoms(mol):
    if not mol:
        return None
    
    for atom in mol.GetAtoms():
        if atom.HasProp('molAtomMapNumber'):
            map_num = atom.GetProp('molAtomMapNumber')
            if map_num in {'1', '2', '3'}:
                atom.SetProp('molAtomMapNumber', '0')
        if atom.GetIsotope() != 0:
            atom.SetIsotope(0)

    return mol

def process_sdf_file(file_path):
    # Read the SDF file
    suppl = Chem.SDMolSupplier(file_path, removeHs=False)
    mols = [mol for mol in suppl if mol is not None]

    # Process each molecule
    processed_mols = [modify_atoms(mol) for mol in mols if mol]

    # Write the processed molecules back to a new SDF file
    output_file = file_path.replace(".sdf", "_processed.sdf")
    writer = Chem.SDWriter(output_file)
    for mol in processed_mols:
        writer.write(mol)
    writer.close()

    # Print out the message indicating the file has been processed
    print(f"File {file_path} has been processed and saved as {output_file}")

# Specific SDF file to process
file_to_process = "BKVP1_Ac_APF_v2_Ac_bridge.sdf"

# Check if the file exists in the current directory
if os.path.exists(file_to_process):
    process_sdf_file(file_to_process)
else:
    print(f"The file {file_to_process} does not exist in the current directory.")

print("Processing complete.")



File BKVP1_Ac_APF_v2_Ac_bridge.sdf has been processed and saved as BKVP1_Ac_APF_v2_Ac_bridge_processed.sdf
Processing complete.


In [5]:
import os
import gc  # Import garbage collection module
from rdkit import Chem
from rdkit.Chem import rdFMCS
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

def load_molecules(file_path, start_index, batch_size=None):
    """Load a batch of molecules from an SDF file starting from a given index."""
    logging.info(f"Loading molecules from {file_path} starting at index {start_index}")
    suppl = Chem.SDMolSupplier(file_path, removeHs=False)
    mols = []

    # Skip molecules up to the start index
    for i, mol in enumerate(suppl):
        if i < start_index:
            continue
        if mol is not None:
            mols.append(mol)
        if batch_size is not None and len(mols) >= batch_size:  # Check batch_size only if it is not None
            break

    logging.info(f"Loaded {len(mols)} molecules starting from index {start_index}")
    return mols

def calculate_substructure_similarity(mol, fragment):
    """Calculate the similarity between a molecule and a fragment."""
    try:
        mcs_result = rdFMCS.FindMCS([mol, fragment], completeRingsOnly=True, ringMatchesRingOnly=True)
        if mcs_result.canceled:
            return 0.0
        mcs = Chem.MolFromSmarts(mcs_result.smartsString)
        if mcs is not None:
            common_atoms = mcs.GetNumHeavyAtoms()
            total_atoms = mol.GetNumHeavyAtoms() + fragment.GetNumHeavyAtoms() - common_atoms
            similarity = common_atoms / total_atoms if total_atoms > 0 else 0.0
            return similarity
        else:
            return 0.0
    except Exception as e:
        logging.error(f"Error in calculating substructure similarity: {e}")
        return 0.0

def find_best_fragment(molecule, fragments):
    """Find the most similar fragment to the given molecule and return its index and similarity."""
    best_similarity = -1
    best_index = -1

    for index, fragment in enumerate(fragments):
        similarity = calculate_substructure_similarity(molecule, fragment)
        if similarity > best_similarity:
            best_similarity = similarity
            best_index = index + 1  # 1-based index

    return best_index, best_similarity

def main():
    fragment_file = "R284_Ac.sdf"
    large_file = "BKVP1_Ac_APF_v2_Ac_bridge_processed_all.sdf"
    output_file = "BKVP1_with_frag_similarity.sdf"
    batch_size = 1000  # Process molecules in batches of 1000
    total_molecules = 75497

    logging.info("Loading fragments...")
    fragments = load_molecules(fragment_file, start_index=0, batch_size=None)
    logging.info(f"Loaded {len(fragments)} fragments.")

    # Initialize the writer
    writer = Chem.SDWriter(output_file)

    # Process molecules in batches, keeping track of the start index
    for start_index in range(0, total_molecules, batch_size):
        logging.info(f"Processing batch starting from index {start_index}...")

        # Load a batch of molecules starting from the current index
        large_mols = load_molecules(large_file, start_index=start_index, batch_size=batch_size)

        for i, mol in enumerate(large_mols):
            best_index, best_similarity = find_best_fragment(mol, fragments)

            # Set the properties for the molecule
            mol.SetProp("BestFragmentIndex", str(best_index))
            mol.SetProp("MaxSimilarity", f"{best_similarity:.4f}")

            # Write the updated molecule to the output file
            writer.write(mol)

            if (start_index + i + 1) % 1000 == 0:
                logging.info(f"Processed {start_index + i + 1}/{total_molecules} molecules.")

        # Force garbage collection to free memory
        del large_mols
        gc.collect()

    logging.info("Processing complete. Output written to file.")
    writer.close()

if __name__ == "__main__":
    main()


2024-09-23 10:26:04,450 - Loading fragments...
2024-09-23 10:26:04,451 - Loading molecules from R284_Ac.sdf starting at index 0
2024-09-23 10:26:04,456 - Loaded 4 molecules starting from index 0
2024-09-23 10:26:04,458 - Loaded 4 fragments.
2024-09-23 10:26:04,552 - Processing batch starting from index 0...
2024-09-23 10:26:04,553 - Loading molecules from BKVP1_Ac_APF_v2_Ac_bridge_processed_all.sdf starting at index 0
2024-09-23 10:26:05,310 - Loaded 1000 molecules starting from index 0
2024-09-23 10:32:09,069 - Processed 1000/75497 molecules.
2024-09-23 10:32:19,375 - Processing batch starting from index 1000...
2024-09-23 10:32:19,376 - Loading molecules from BKVP1_Ac_APF_v2_Ac_bridge_processed_all.sdf starting at index 1000
2024-09-23 10:32:20,640 - Loaded 1000 molecules starting from index 1000
2024-09-23 10:39:20,864 - Processed 2000/75497 molecules.
2024-09-23 10:39:20,964 - Processing batch starting from index 2000...
2024-09-23 10:39:20,965 - Loading molecules from BKVP1_Ac_APF

In [1]:
from rdkit import Chem
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

def load_molecules(file_path):
    """Load molecules from an SDF file with their properties."""
    logging.info(f"Loading molecules from {file_path}")
    suppl = Chem.SDMolSupplier(file_path, removeHs=False)
    mols = []
    for mol in suppl:
        if mol is not None:
            mols.append(mol)
    logging.info(f"Loaded {len(mols)} molecules from {file_path}")
    return mols

def sort_molecules_by_fragment_and_sum(molecules):
    """Sort molecules by BestFragmentIndex and Sum."""
    sorted_molecules = {1: [], 2: [], 3: [], 4: []}
    for mol in molecules:
        try:
            best_fragment_index = int(mol.GetProp("BestFragmentIndex"))
            sum_value = float(mol.GetProp("Sum"))  # Assuming "Sum" is stored as a property
            sorted_molecules[best_fragment_index].append((sum_value, mol))
        except Exception as e:
            logging.warning(f"Skipping a molecule due to missing or invalid properties: {e}")

    # Sort each list by the 'Sum' value
    for index in sorted_molecules:
        sorted_molecules[index].sort(key=lambda x: x[0])

    return sorted_molecules

def reorder_and_write_molecules(sorted_molecules, output_file):
    """Reorder molecules and write them to a new SDF file in the desired order."""
    logging.info(f"Writing reordered molecules to {output_file}")
    writer = Chem.SDWriter(output_file)
    
    index = 0
    count = 0
    while True:
        added_any = False
        # Cycle through fragment indices 1, 2, 3, 4
        for fragment_index in [1, 2, 3, 4]:
            if index < len(sorted_molecules[fragment_index]):
                _, mol = sorted_molecules[fragment_index][index]
                writer.write(mol)
                count += 1
                added_any = True

        index += 1
        
        if not added_any:  # No molecules left to add
            break

    logging.info(f"Total molecules written: {count}")
    writer.close()

def main():
    # Specify file names assuming they are in the same directory as the script
    input_file = "BKVP1_with_frag_similarity.sdf"
    output_file = "BKVP1_reordered.sdf"

    # Load molecules from the input file
    molecules = load_molecules(input_file)

    # Sort molecules by BestFragmentIndex and Sum
    sorted_molecules = sort_molecules_by_fragment_and_sum(molecules)

    # Reorder molecules in 1, 2, 3, 4 sequence based on Sum and write to new file
    reorder_and_write_molecules(sorted_molecules, output_file)

    logging.info("Reordering complete.")

if __name__ == "__main__":
    main()


2024-09-24 00:10:06,017 - Loading molecules from BKVP1_with_frag_similarity.sdf
2024-09-24 00:10:51,014 - Loaded 75497 molecules from BKVP1_with_frag_similarity.sdf
2024-09-24 00:10:51,206 - Writing reordered molecules to BKVP1_reordered.sdf
2024-09-24 00:11:33,296 - Total molecules written: 75497
2024-09-24 00:11:33,297 - Reordering complete.


In [1]:
from rdkit import Chem
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

def load_molecules(file_path):
    """Load molecules from an SDF file with their properties."""
    logging.info(f"Loading molecules from {file_path}")
    suppl = Chem.SDMolSupplier(file_path, removeHs=False)
    mols = []
    for mol in suppl:
        if mol is not None:
            mols.append(mol)
    logging.info(f"Loaded {len(mols)} molecules from {file_path}")
    return mols

def sort_molecules_by_fragment_and_similarity(molecules):
    """Sort molecules by BestFragmentIndex, prioritize MaxSimilarity (descending), then Sum (ascending)."""
    sorted_molecules = {1: [], 2: [], 3: [], 4: []}
    
    for mol in molecules:
        try:
            best_fragment_index = int(mol.GetProp("BestFragmentIndex"))
            max_similarity = float(mol.GetProp("MaxSimilarity"))  # Assuming "MaxSimilarity" is stored as a property
            sum_value = float(mol.GetProp("Sum"))  # Assuming "Sum" is stored as a property
            sorted_molecules[best_fragment_index].append((max_similarity, sum_value, mol))
        except Exception as e:
            logging.warning(f"Skipping a molecule due to missing or invalid properties: {e}")

    # Sort each list first by 'MaxSimilarity' (descending) and then by 'Sum' (ascending)
    for index in sorted_molecules:
        sorted_molecules[index].sort(key=lambda x: (-x[0], x[1]))

    return sorted_molecules

def reorder_and_write_molecules(sorted_molecules, output_file):
    """Reorder molecules and write them to a new SDF file in the desired order."""
    logging.info(f"Writing reordered molecules to {output_file}")
    writer = Chem.SDWriter(output_file)
    
    index = 0
    count = 0
    while True:
        added_any = False
        # Cycle through fragment indices 1 through 10
        for fragment_index in range(1, 5):
            if index < len(sorted_molecules[fragment_index]):
                _, _, mol = sorted_molecules[fragment_index][index]
                writer.write(mol)
                count += 1
                added_any = True

        index += 1
        
        if not added_any:  # No molecules left to add
            break

    logging.info(f"Total molecules written: {count}")
    writer.close()

def main():
    # Specify file names assuming they are in the same directory as the script
    input_file = "BKVP1_with_frag_similarity.sdf"
    output_file = "BKVP1_reordered.sdf"

    # Load molecules from the input file
    molecules = load_molecules(input_file)

    # Sort molecules by BestFragmentIndex, MaxSimilarity, and Sum
    sorted_molecules = sort_molecules_by_fragment_and_similarity(molecules)

    # Reorder molecules in 1, 2, 3, 4 sequence based on MaxSimilarity and Sum, and write to new file
    reorder_and_write_molecules(sorted_molecules, output_file)

    logging.info("Reordering complete.")

if __name__ == "__main__":
    main()


2024-09-27 05:07:26,037 - Loading molecules from BKVP1_with_frag_similarity.sdf
2024-09-27 05:08:11,831 - Loaded 75497 molecules from BKVP1_with_frag_similarity.sdf
2024-09-27 05:08:12,215 - Writing reordered molecules to BKVP1_reordered.sdf
2024-09-27 05:08:53,458 - Total molecules written: 75497
2024-09-27 05:08:53,460 - Reordering complete.


In [2]:
from rdkit import Chem

# Function to read SDF file and return a dictionary of molecules with their identifiers
def read_sdf(file_path):
    print(f"Reading SDF file: {file_path}")
    suppl = Chem.SDMolSupplier(file_path)
    mol_dict = {}
    for mol in suppl:
        if mol is not None:
            # Create a tuple identifier from the properties
            identifier = (
                mol.GetProp('synton_id_1'), 
                mol.GetProp('synton_id_2'), 
                mol.GetProp('synton_id_3'), 
                mol.GetProp('rxn_ID'), 
                mol.GetProp('const_synth')
            )
            mol_dict[identifier] = mol
    print(f"Completed reading SDF file: {file_path}")
    return mol_dict

# Function to update structures in the fragment file with the corresponding structures from the parent file
def update_structures(fragment_file, parent_file, output_file):
    print(f"Updating structures in the fragment file '{fragment_file}' with data from '{parent_file}'...")

    # Read molecules from both SDF files
    frag_mols = read_sdf(fragment_file)
    parent_mols = read_sdf(parent_file)

    updated_mols = []

    # Replace fragment structures with parent structures if the identifier matches
    for identifier, frag_mol in frag_mols.items():
        if identifier in parent_mols:
            parent_mol = parent_mols[identifier]
            
            # Copy 'BestFragmentIndex' and 'MaxSimilarity' from frag_mol to parent_mol
            if frag_mol.HasProp("BestFragmentIndex"):
                parent_mol.SetProp("BestFragmentIndex", frag_mol.GetProp("BestFragmentIndex"))
            if frag_mol.HasProp("MaxSimilarity"):
                parent_mol.SetProp("MaxSimilarity", frag_mol.GetProp("MaxSimilarity"))

            frag_mols[identifier] = parent_mol  # Replace structure with the parent structure

    # Collect the updated molecules and ensure properties are preserved
    for identifier, mol in frag_mols.items():
        for i, prop in enumerate(['synton_id_1', 'synton_id_2', 'synton_id_3', 'rxn_ID', 'const_synth']):
            mol.SetProp(prop, identifier[i])  # Ensure the identifier properties are preserved
        updated_mols.append(mol)

    # Write the updated molecules to the new SDF file
    writer = Chem.SDWriter(output_file)
    for mol in updated_mols:
        writer.write(mol)
    writer.close()
    
    print(f"Completed updating structures. The updated SDF file is saved as: {output_file}")

# Paths to the fragment and parent SDF files
fragment_file = 'BKVP1_reordered.sdf'
parent_file = 'BKVP1_Ac_APF_v2_Ac_bridge.sdf'
output_file = 'updated_BKVP1_reordered.sdf'

# Update the structures
update_structures(fragment_file, parent_file, output_file)



Updating structures in the fragment file 'BKVP1_reordered.sdf' with data from 'BKVP1_Ac_APF_v2_Ac_bridge.sdf'...
Reading SDF file: BKVP1_reordered.sdf
Completed reading SDF file: BKVP1_reordered.sdf
Reading SDF file: BKVP1_Ac_APF_v2_Ac_bridge.sdf
Completed reading SDF file: BKVP1_Ac_APF_v2_Ac_bridge.sdf
Completed updating structures. The updated SDF file is saved as: updated_BKVP1_reordered.sdf
